# Visualization of Data from Formspring 

Involves "smart" parsing, specifically: convert words into sounds and then back into a dictionary to see if we can decipher some of the slang used in online communication. 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import random
import sys
import os
from util.Util import *
from util.parser import formspring_data_parser
import pandas as pd
from sklearn import model_selection

In [3]:
import sys
sys.path.append("/Users/luisd/dev/cyberbullying-detection")
# from nn import *
# import nn 
# import fc_nn 

In [4]:
# sys.path.append(...) # in case we want to add something else

In [5]:
import phonemes_from_graphemes as pg
from words_2_vectors import * 
from util.parser import formspring_data_parser, wikipedia_talk_parser

In [6]:
# import importlib
# importlib.reload(pg)
from phonemes_from_graphemes import *

In [7]:
import pandas as pd

In [8]:
from collections import Counter
import numpy as np
from tqdm import tqdm 

### Where am I expecting to see the data

In [9]:
data_dir = "./data"

In [10]:
common_logger = get_logger(name = "common_logger_2", debug_log_file_name = "common_logger_2.log")

2017-06-19 15:04:04,759 - common_logger_2 - INFO [Util.get_logger:55 => 'common_logger_2': logging 'INFO'+ logs to Console, 'DEBUG'+ logs to '/Users/luisd/dev/cyberbullying-detection/common_logger_2.log']


Logger created
Creating debug handler at '/Users/luisd/dev/cyberbullying-detection/common_logger_2.log'
'common_logger_2': logging 'INFO'+ logs to Console, 'DEBUG'+ logs to '/Users/luisd/dev/cyberbullying-detection/common_logger_2.log'


In [11]:
common_logger.handlers[1].baseFilename

'/Users/luisd/dev/cyberbullying-detection/common_logger_2.log'

In [29]:
from util.parser.formspring_data_parser import * 
from util.parser.wikipedia_talk_parser import *

In [13]:
# a_formspring_data_parser = Formspring_Data_Parser(all_data = pd.read_csv("really_all_of_them.csv"), alogger = common_logger)

## Formspring Data Parser: Load or build from scratch 

### If you want to LOAD it, set flag to True; otherwise, set it to False   

In [15]:
formspring_parser_load = True 

### if there are components (ModelWrapper, Sounds Dict and/or Phonemes Lib. already present, set the flag below to True 

In [16]:
existing_components = False 

In [17]:
print("Please monitor the debug file (ie, run 'tail -f {}')".format(common_logger.handlers[1].baseFilename))


Please monitor the debug file (ie, run 'tail -f /Users/luisd/dev/cyberbullying-detection/common_logger_2.log')


In [19]:
(a_formspring_data_parser, mw, sounds_dict, phonemesFactory) = formspring_load_or_create(
    formspring_parser_load, 
    common_logger, 
    mw if existing_components else None, 
    sounds_dict if existing_components else None, 
    phonemesFactory if existing_components else None, 
    data_dir)

Please monitor the debug file (ie, run 'tail -f /Users/luisd/dev/cyberbullying-detection/common_logger_2.log')


2017-06-19 15:04:58,072 - common_logger_2 - INFO [formspring_data_parser.formspring_load_or_create:188 => Formspring parser should have been loaded]


## Commenting all code subsumed by call above ^ 
### Left it in case we want to revive it for sanity-checking 

## Load Word2Vec model trained with Google News 

In [ ]:
# mw = ModelWrapper.from_google_news_model(data_dir=data_dir, alogger=common_logger)

In [ ]:
# m = mw.model # cache model definition

In [ ]:
# len(m.vocab)

## Creation of Sounds' Dictionary  

### Let's read the sounds' dictionary
(and minimally sanit-check it) 

In [ ]:
# sounds_dict = SoundsDict(a_dir=data_dir, alogger=common_logger)

In [ ]:
# sounds_dict['ju:']

In [ ]:
# mw.set_sounds_dict(sounds_dict=sounds_dict)

In [ ]:
# mw.sound_to_word('ju:')

### If you want to LOAD it, set flag to True; otherwise, set it to False   

In [26]:
wiki_parser_load = True 

In [27]:
existing_components = (mw is not None or sounds_dict is not None or phonemesFactory is not None) 

In [30]:
(wiki_data_parser, mw, sounds_dict, phonemesFactory) = wikinastigrams_load_or_create(
        wiki_parser_load,
        common_logger,
        mw if existing_components else None,
        sounds_dict if existing_components else None,
        phonemesFactory if existing_components else None,
        data_dir)

Please monitor the debug file (ie, run 'tail -f /Users/luisd/dev/cyberbullying-detection/common_logger_2.log')


2017-06-19 15:13:19,742 - common_logger_2 - INFO [wikipedia_talk_parser.wikinastigrams_load_or_create:143 => Wiki parser should have been loaded]


In [20]:
# questions_answers = ["{}; {}".format(q, a) for q, a in list(zip(really_all['question'].tolist(), really_all['answer'].tolist()))]
questions_answers_formspring = a_formspring_data_parser.interactions()
labels_formspring = a_formspring_data_parser.labels()

In [21]:
questions_answers_formspring[0:10] 

["what's your favorite song laughter; I like too many songs to have a favorite",
 '----------------------------------------------------------_3; haha jk',
 "'3; haha jk",
 'angel you duh; Really Thanks haha',
 'nan; nan',
 'nan; nan',
 'any makeup tips i suck at doing my makeup laughter; Sure Like tell me wht u know Like wht do you use',
 "I miss It's Emma btw haha; EMMA laughter I yu",
 "I miss It's Emma btw haha; EMMA laughter I ue",
 "I miss It's Emma btw haha; EMMA laughter I ew"]

In [22]:
labels_formspring[0:10]

['CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN']

In [23]:
import nn 
import fc_nn 

Using TensorFlow backend.


In [24]:
cyber_fc = fc_nn.CyberbullyingFullyConnectedNetwork(reviews=questions_answers_formspring, labels=labels_formspring,hidden_nodes=300,learning_rate=0.01)

In [25]:
cyber_fc.train(questions_answers_formspring, labels_formspring)

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:6.77% Speed(reviews/sec):5753. #Correct:2339 #Trained:2501 Training Accuracy:93.5%
Progress:13.5% Speed(reviews/sec):5827. #Correct:4784 #Trained:5001 Training Accuracy:95.6%
Progress:20.3% Speed(reviews/sec):6028. #Correct:7077 #Trained:7501 Training Accuracy:94.3%
Progress:27.1% Speed(reviews/sec):6139. #Correct:9484 #Trained:10001 Training Accuracy:94.8%
Progress:33.8% Speed(reviews/sec):5740. #Correct:11828 #Trained:12501 Training Accuracy:94.6%
Progress:40.6% Speed(reviews/sec):5018. #Correct:14262 #Trained:15001 Training Accuracy:95.0%
Progress:47.4% Speed(reviews/sec):4143. #Correct:16723 #Trained:17501 Training Accuracy:95.5%
Progress:54.2% Speed(reviews/sec):4338. #Correct:19028 #Trained:20001 Training Accuracy:95.1%
Progress:61.0% Speed(reviews/sec):4373. #Correct:21467 #Trained:22501 Training Accuracy:95.4%
Progress:67.7% Speed(reviews/sec):4391. #Correct:23901 #Trained:25001 Training 

In [31]:
interactions_wiki = wiki_data_parser.interactions()
labels_wiki = wiki_data_parser.labels()
print("There are {} data points".format(len(interactions_wiki)))

There are 15202772 data points


In [ ]:
interactions_wiki[0:10] 

In [ ]:
def safe_check_nan(s) -> bool:
    try:
        return math.isnan(float(s))
    except:
        return False 

In [ ]:
safe_check_nan("1.0")

In [ ]:
all_nans = list(filter(lambda x: safe_check_nan(x), interactions_wiki))

In [ ]:
len(all_nans)

# TODO: filter 'nans' (see ABOVE) 

In [ ]:
labels_wiki[0:10]

In [ ]:
str(float("nan"))

In [32]:
def positive_negative_total_counts(labels, questions_answers):
    # Create three Counter objects to store positive, negative and total counts
    positive_counts = Counter()
    negative_counts = Counter()
    total_counts = Counter()
    for label, review in tqdm(zip(labels, questions_answers), total=len(labels)):
        # print("===========> label = '{}', review = '{}'".format(label, review))
        try:
            words = str(review).split(' ') 
            if label == 'THREAT':
                for word in words:
                    negative_counts[word] += 1
            else:
                for word in words:
                    positive_counts[word] += 1
            for word in words:
                total_counts[word] += 1
        except Exception as e:
            print("ERROR => problem with label = '{}', review = '{}'".format(label, review))
            raise e

    pos_neg_ratios = Counter()

    unique_words = total_counts.keys()
    for word in unique_words:
        if total_counts[word] >= 100:
            pos_neg_ratios[word] = positive_counts[word] / float(negative_counts[word]+1)


    unique_words = pos_neg_ratios.keys()# set(pos_neg_ratios.elements())
    for word in unique_words:
        if (pos_neg_ratios[word] > 1):
            pos_neg_ratios[word] = np.log(pos_neg_ratios[word])
        else:
            pos_neg_ratios[word] = -np.log(1/(pos_neg_ratios[word] + 0.01))     
    return (positive_counts, negative_counts, total_counts, pos_neg_ratios)

## Let's examine Formspring Data 

In [33]:
(positive_counts_formspring, negative_counts_formspring, total_counts_formspring, pos_neg_ratios_formspring) = positive_negative_total_counts(labels = labels_formspring, questions_answers = questions_answers_formspring)

100%|██████████| 36885/36885 [00:01<00:00, 21688.56it/s]


In [34]:
negative_counts_formspring.most_common()

[('a', 6752),
 ('you', 6372),
 ('u', 6204),
 ('i', 5824),
 ('ur', 4956),
 ('at', 4873),
 ('threw', 3600),
 ('your', 3416),
 ('no', 3104),
 ('shit', 3101),
 ('hair', 3004),
 ('some', 2953),
 ('the', 2916),
 ('be', 2665),
 ('are', 2608),
 ('mah', 2600),
 ('was', 2577),
 ('then', 2572),
 ('would', 2534),
 ('if', 2532),
 ('say', 2519),
 ('head', 2501),
 ('girl', 2467),
 ('people', 2456),
 ('UR', 2453),
 ('stfu', 2451),
 ('make', 2443),
 ('good', 2435),
 ('saw', 2425),
 ('fat', 2424),
 ('mad', 2414),
 ('kno', 2413),
 ('sense', 2407),
 ('of;', 2402),
 ('named', 2401),
 ('corndog', 2400),
 ('CORNDOG', 2400),
 ('GOOR', 2400),
 ('twinkie', 2400),
 ('bitch', 1795),
 ('bich', 1456),
 ('her', 1364),
 ("didn't", 1224),
 ("did'nt", 1220),
 ('thru', 1200),
 ('off', 1047),
 ('fair', 961),
 ('phair', 960),
 ('phare', 960),
 ('fare', 960),
 ('faire', 960),
 ('ass', 853),
 ('lick', 807),
 ('lik', 804),
 ('lic', 803),
 ('on', 795),
 ('hoe', 771),
 ('so', 766),
 ('pussy', 727),
 ('love', 685),
 ('too', 638

In [35]:
# negative_counts.most_common()
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios_formspring["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios_formspring["amazing"]))
print("Pos-to-neg ratio for 'shit' = {}".format(pos_neg_ratios_formspring["shit"]))

Pos-to-neg ratio for 'the' = 2.461319014418187
Pos-to-neg ratio for 'amazing' = 5.746203190540153
Pos-to-neg ratio for 'shit' = -2.5972980568759496


### What are the most common words for 'threats' and 'non-threats' 

In [36]:
pos_neg_ratios_formspring.most_common()

[('rock', 9.3969859002501916),
 ('badr', 8.7021778656296753),
 ('spend', 8.4314174143948328),
 ('Baby', 8.4213428657594029),
 ('glue', 8.4144957931778954),
 ('bed', 8.1028891346408685),
 ('Nicki', 8.0100275284817339),
 ('andr', 8.009363076630045),
 ('Young', 8.009363076630045),
 ('bakr', 8.009363076630045),
 ('rokr', 8.0090306850697299),
 ('love;', 7.640603826393634),
 ('money', 7.4073177104694174),
 ('gorgeous', 7.3895639536776354),
 ('walk', 7.3821243657375124),
 ('running', 7.3777589082278725),
 ('spoke', 7.3740018593501606),
 ('cold', 7.3607399030582776),
 ('hold', 7.3581937527330323),
 ('along', 7.3581937527330323),
 ('photo', 7.357556200910353),
 ('rest', 7.3562798765507482),
 ('room', 7.3485875309275928),
 ('race', 7.3466551631765391),
 ('lover', 7.3453648404168685),
 ('attractive', 7.3401868353201154),
 ('fly', 7.3401868353201154),
 ('sing', 7.333676395657684),
 ('lights', 7.3297496890415124),
 ('double', 7.329093736246592),
 ('Michael', 7.3251489579555749),
 ('annoys', 7.32514

In [37]:
list(reversed(pos_neg_ratios_formspring.most_common()))[0:30]

[('ol', -4.6051701859880918),
 ('matta', -4.6051701859880918),
 ('dawk', -4.6051701859880918),
 ('wiht', -4.6051701859880918),
 ('priscilla', -4.6051701859880918),
 ('prob', -4.6051701859880918),
 ('hare', -4.6051701859880918),
 ('twinkie', -4.6051701859880918),
 ('haire', -4.6051701859880918),
 ('GOOR', -4.6051701859880918),
 ('CORNDOG', -4.6051701859880918),
 ('corndog', -4.6051701859880918),
 ('hopp', -4.6051701859880918),
 ('alexis;', -4.6051701859880918),
 ('ghetto', -4.6051701859880918),
 ('threw', -4.2969425047274195),
 ('otha', -4.2388078038936694),
 ('stfu', -4.2067992413293345),
 ('bea', -4.1653125479201716),
 ('beah', -4.1653125479201716),
 ('yao', -4.1531850622450337),
 ('yow', -4.1531850622450337),
 ('yau', -4.1531850622450337),
 ('thru', -4.1459446655796457),
 ('mutha', -4.1047885213831838),
 ('kus', -3.9913597476646672),
 ('fade', -3.9753926193607354),
 ('UR', -3.9439031633388089),
 ('wdf', -3.9169857947702749),
 ('ah;', -3.9169857947702749)]

### Yeah... doesn't look like there is a pattern here... 
Maybe some more work on the pre-treatment is needed 

### Let's take a look 

In [38]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [40]:
hist, edges = np.histogram(list(map(lambda x:x[1],pos_neg_ratios_formspring.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Word Positive/Negative Affinity Distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [42]:
frequency_frequency = Counter()

for word, cnt in total_counts_formspring.most_common():
    frequency_frequency[cnt] += 1

In [43]:
hist, edges = np.histogram(list(map(lambda x:x[1],frequency_frequency.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="The frequency distribution of the words in our corpus")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [44]:
# vocab = set(total_counts.keys())
# vocab_size = len(vocab)
# print(vocab_size)

In [45]:
len(labels_wiki)

15202772

In [46]:
len(interactions_wiki)

15202772

## Let's examine Wiki Data 

In [47]:
# ffrom = 420
# fto = 430
# (positive_counts_wiki, negative_counts_wiki, total_counts_wiki, pos_neg_ratios_wiki) = positive_negative_total_counts(labels = labels_wiki[ffrom:fto], questions_answers = interactions_wiki[ffrom:fto])
(positive_counts_wiki, negative_counts_wiki, total_counts_wiki, pos_neg_ratios_wiki) = positive_negative_total_counts(labels = labels_wiki, questions_answers = interactions_wiki)

100%|██████████| 15202772/15202772 [03:55<00:00, 64580.04it/s]


In [48]:
negative_counts_wiki.most_common()

[('YOU', 19133459),
 ('fac', 6908843),
 ('facc', 6908843),
 ('phac', 6908843),
 ('@', 588266),
 ('u', 103302),
 ('yu', 102652),
 ('ew', 102645),
 ('yoo', 102644),
 ('yue', 102644),
 ('ue', 102644),
 ('ewe', 102644),
 ('==', 22790),
 ('you', 18366),
 ('I', 11752),
 ('of', 10000),
 ('a', 6911),
 ('the', 6468),
 ('during', 6286),
 ('ewes', 6272),
 ('to', 6084),
 ('is', 4428),
 ('following', 3171),
 ('that', 3159),
 ('are', 3150),
 ('threw', 3140),
 ('thru', 3137),
 ('folowing', 3137),
 ('your', 3010),
 ('my', 2645),
 ('You', 2362),
 ('in', 1987),
 ('it', 1982),
 ('this', 1798),
 ('an', 1729),
 ('on', 1618),
 ('fuck', 1594),
 ('toe', 1575),
 ('toh', 1569),
 ('tow', 1569),
 ('towe', 1569),
 ('not', 1568),
 ('for', 1532),
 ('and', 1518),
 ('me', 1507),
 ('Assad', 1504),
 ('have', 1365),
 ('one', 1269),
 ('who', 1201),
 ('about', 1188),
 ('with', 1187),
 ('i', 1166),
 ('single', 1124),
 ('be', 1100),
 ('fucking', 1097),
 ('THE', 1091),
 ('or', 1077),
 ('do', 1062),
 ('like', 1002),
 ('idiot',

In [49]:
# negative_counts.most_common()
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios_wiki["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios_wiki["amazing"]))
print("Pos-to-neg ratio for 'shit' = {}".format(pos_neg_ratios_wiki["shit"]))

Pos-to-neg ratio for 'the' = 6.9553355749411825
Pos-to-neg ratio for 'amazing' = 0
Pos-to-neg ratio for 'shit' = -1.6037198851175392


### What are the most common words for 'threats' and 'non-threats' 

In [50]:
pos_neg_ratios_wiki.most_common()

[('january', 15.01618984461023),
 (';/', 14.668135999872536),
 ('slash', 14.556910540295222),
 ("What's", 12.243623997324812),
 ('start', 11.97173693308685),
 ('ship', 11.562010823660708),
 ('schip', 11.561553711269765),
 ('shipp', 11.561553711269765),
 ('wrong', 10.881608541509278),
 ('person', 10.881144127924619),
 ('----------_-----------------------------------', 10.760934922430392),
 ('----------_-----------', 10.760934922430392),
 ('----------_---', 10.760934922430392),
 ('-----------', 10.760934922430392),
 ('----------_---------------------------------', 10.760934922430392),
 ('-------------', 10.760934922430392),
 ('-----------------------------------------------------', 10.760934922430392),
 ('-----------------------------', 10.760934922430392),
 ('---------------------------------------', 10.760934922430392),
 ('---------', 10.760934922430392),
 ('---------------------------------', 10.760934922430392),
 ('---------------------------------------------------------',
  10.7609

In [51]:
list(reversed(pos_neg_ratios_wiki.most_common()))[0:30]

[('FUCK', -4.6051701859880918),
 ('NIGGER', -4.6051701859880918),
 ('phac', -4.6048228654754979),
 ('facc', -4.6048228654754979),
 ('fac', -4.6048083964050166),
 ('YOU', -4.6025551585903353),
 ('@', -4.5864770141435827),
 ('ewes', -4.5584545458212586),
 ('folowing', -4.5433840150592886),
 ('gov', -4.3696205101337826),
 ('Assad', -4.369481075610266),
 ('threw', -4.3287741019811703),
 ('ewe', -4.2966272858408212),
 ('yue', -4.2966272858408212),
 ('ew', -4.2959145441636828),
 ('yoo', -4.2959119526438956),
 ('ue', -4.2944828195270253),
 ('yu', -4.2937898715310832),
 ('FUCKING', -4.2890849215494029),
 ('Loser', -4.2415022065583887),
 ('tuy', -4.193355850201268),
 ('tye', -4.193355850201268),
 ('tai', -4.193355850201268),
 ('pty', -4.193355850201268),
 ('SUCK', -4.1767965870830572),
 ('ti', -4.113329226809368),
 ('thai', -4.1123399064411794),
 ('Fuck', -4.0269968789318371),
 ('thru', -4.0192264476070116),
 ('beaked', -3.9774352706143139)]

### mmh...? 

### Let's take a look 

In [52]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [53]:
hist, edges = np.histogram(list(map(lambda x:x[1],pos_neg_ratios_wiki.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Word Positive/Negative Affinity Distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [54]:
frequency_frequency = Counter()

for word, cnt in total_counts_wiki.most_common():
    frequency_frequency[cnt] += 1

In [55]:
hist, edges = np.histogram(list(map(lambda x:x[1],frequency_frequency.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="The frequency distribution of the words in our corpus")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)